In [ ]:
import pandas as pd
import uuid
import time
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.http import models

# 1. CREDENCIALES (LLENAR AQUÍ)

OPENAI_API_KEY = "sk-proj-AGvGFTE0nNy49w087VkfNJSMHWJzcCfRDH8bOwa0"
QDRANT_URL = "https://08a32.us-east4-0.gcp.cloud.qdrant.io"
QDRANT_API_KEY = "eXVCJ9.eyJhY2Nlc3MiOiJtIn0.lj72v9z_Tns"


# 2. CONFIGURACIÓN TÉCNICA

COLLECTION_NAME = "prueba_tecnica"
MODELO_EMBEDDING = "text-embedding-3-small" 
DIMENSION_VECTOR = 1536 

# Inicializamos los clientes
client_openai = OpenAI(api_key=OPENAI_API_KEY)
client_qdrant = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

print("Clientes configurados correctamente.")

Clientes configurados correctamente.


In [11]:
# ============================================
# DIAGNÓSTICO: Identificar casos problemáticos
# ============================================

print("Analizando todos los casos...")
print(f"Total de casos en DataFrame: {len(df)}\n")

# Analizar longitud de todos los textos
df['longitud_texto'] = df['texto_vectorizar'].str.len()
df['longitud_tokens_estimada'] = df['longitud_texto'] / 4  # Estimación: 1 token ≈ 4 chars

# Encontrar casos problemáticos
casos_problematicos = df[df['longitud_tokens_estimada'] > 8000].copy()

print(f"Estadísticas de longitud:")
print(f"  - Promedio: {df['longitud_texto'].mean():.0f} caracteres")
print(f"  - Máximo: {df['longitud_texto'].max():.0f} caracteres")
print(f"  - Mínimo: {df['longitud_texto'].min():.0f} caracteres")
print(f"\nCasos que exceden 8000 tokens (~32,000 chars): {len(casos_problematicos)}")

if len(casos_problematicos) > 0:
    print("\nCASOS PROBLEMÁTICOS:")
    for idx, row in casos_problematicos.iterrows():
        print(f"\n  Fila {idx}:")
        print(f"    - Providencia: {row.get('Providencia', 'N/A')}")
        print(f"    - Longitud: {row['longitud_texto']:,} caracteres")
        print(f"    - Tokens estimados: {row['longitud_tokens_estimada']:.0f}")
        print(f"    - Tema: {row.get('Tema - subtema', 'N/A')[:100]}...")

🔍 Analizando todos los casos...
Total de casos en DataFrame: 329

Estadísticas de longitud:
  - Promedio: 4990 caracteres
  - Máximo: 35614 caracteres
  - Mínimo: 937 caracteres

Casos que exceden 8000 tokens (~32,000 chars): 1

CASOS PROBLEMÁTICOS:

  Fila 321:
    - Providencia: SU.546/23
    - Longitud: 35,614 caracteres
    - Tokens estimados: 8904
    - Tema: No disponible...


In [2]:
def get_embedding(text, model=MODELO_EMBEDDING):
    """Genera el vector numérico usando la API de OpenAI"""
    text = text.replace("\n", " ") # Limpieza básica recomendada por OpenAI
    return client_openai.embeddings.create(input=[text], model=model).data[0].embedding

# Prueba rápida para ver si la llave funciona
try:
    test_vec = get_embedding("Prueba de conexión")
    print(f"Conexión con OpenAI exitosa. Dimensión del vector: {len(test_vec)}")
except Exception as e:
    print(f"Error conectando con OpenAI: {e}")

Conexión con OpenAI exitosa. Dimensión del vector: 1536


In [22]:
# 1. Leer Excel
try:
    df = pd.read_excel('Datos/sentencias_pasadas.xlsx', engine='openpyxl') # Ajusta nombre si es necesario
    df = df.fillna("No disponible")
    print(f"Datos cargados: {len(df)} registros.")
except Exception as e:
    print(f"Error leyendo archivo: {e}")

# 2. Crear 'Super Texto' para contexto
# Concatenamos TEMA + SINTESIS + RESUELVE
df['texto_vectorizar'] = (
    "TEMA: " + df['Tema - subtema'].astype(str) + ". " +
    "HECHOS DEL CASO: " + df['sintesis'].astype(str) + ". " +
    "DECISIÓN DEL JUEZ: " + df['resuelve'].astype(str)
)

print("Ejemplo de texto a vectorizar:")
print("-" * 50)
print(df['texto_vectorizar'].iloc[99])

Datos cargados: 329 registros.
Ejemplo de texto a vectorizar:
--------------------------------------------------
TEMA: No disponible. HECHOS DEL CASO: La accionante, actuó como representante legal de una hija que experimentó acoso escolar y fue diagnosticada con depresión y ansiedad y que derivó en una ideación suicida. La vulneración de derechos fundamentales se atribuye al hecho de que la institución educativa no desplegó ninguna acción para contener la precitada circunstancia y para proteger a la adolescente. Igualmente, porque trazaron un Plan Individualizado de Ajustes Razonables (PIAR) que no tuvo en cuenta las condiciones y necesidades académicas especiales que requería la joven, sino que se diseñó únicamente con el propósito de dar cumplimiento formal a la ley y no con la intención de garantizar los derechos fundamentales de la adolescente, lo cual condujo a que ésta reprobara el año escolar y a que no se le permitiera realizar la nivelación correspondiente. Así mismo, porque n

In [12]:
import time
from qdrant_client import models
import uuid

# ============================================
# CONFIGURACIÓN DE CHUNKING
# ============================================
MAX_CHARS_POR_CHUNK = 30000  # ~7,500 tokens (seguro para OpenAI)
OVERLAP_CHARS = 3000         # 3k de overlap para mantener contexto

def crear_chunks_inteligentes(texto, max_chars=MAX_CHARS_POR_CHUNK, overlap=OVERLAP_CHARS):
    """Divide texto largo en chunks con overlap"""
    if len(texto) <= max_chars:
        return [texto]
    
    chunks = []
    inicio = 0
    
    while inicio < len(texto):
        fin = inicio + max_chars
        
        # Buscar punto de corte natural (punto, salto, espacio)
        if fin < len(texto):
            for separador in ['. ', '\n', ' ']:
                posicion = texto.rfind(separador, inicio, fin)
                if posicion > inicio + (max_chars * 0.8):
                    fin = posicion + len(separador)
                    break
        
        chunks.append(texto[inicio:fin])
        inicio = fin - overlap
        
        if inicio >= len(texto):
            break
    
    return chunks


# ============================================
# PROCESAMIENTO CON CHUNKING
# ============================================
print(f"1. Reiniciando colección: {COLLECTION_NAME}...")

if client_qdrant.collection_exists(collection_name=COLLECTION_NAME):
    client_qdrant.delete_collection(collection_name=COLLECTION_NAME)
    print("   -> Versión anterior eliminada.")

client_qdrant.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=DIMENSION_VECTOR,
        distance=models.Distance.COSINE
    )
)
print("   -> Colección creada vacía.\n")

print("2. Procesando documentos con CHUNKING INTELIGENTE...")
print(f"   - Max chars por chunk: {MAX_CHARS_POR_CHUNK:,}")
print(f"   - Overlap: {OVERLAP_CHARS:,} caracteres\n")

points = []
casos_procesados = 0
total_chunks = 0
casos_divididos = []
errores = []

for idx, row in df.iterrows():
    
    texto_original = row['texto_vectorizar']
    chunks = crear_chunks_inteligentes(texto_original)
    
    # Registrar si se dividió
    if len(chunks) > 1:
        casos_divididos.append({
            "fila": idx,
            "providencia": row.get('Providencia', 'N/A'),
            "longitud_original": len(texto_original),
            "num_chunks": len(chunks)
        })
        print(f"  Fila {idx} ({row.get('Providencia', 'N/A')}): {len(chunks)} chunks")
    
    # Procesar cada chunk
    for chunk_num, chunk_texto in enumerate(chunks, 1):
        try:
            # Generar embedding
            vector = get_embedding(chunk_texto)
            
            # Payload con información del chunk
            payload = {
                # Datos del caso original
                "id_caso": row.get('#', idx),
                "fila_excel": idx,
                "providencia": row.get('Providencia', 'N/A'),
                "fecha": str(row.get('Fecha Sentencia', 'N/A')),
                "tema": row.get('Tema - subtema', 'N/A'),
                
                # Metadata COMPLETA (sin truncar)
                "sintesis": str(row.get('sintesis', 'N/A')),
                "resuelve": str(row.get('resuelve', 'N/A')),
                
                # Info del chunking
                "es_chunk": len(chunks) > 1,
                "chunk_num": chunk_num,
                "total_chunks": len(chunks),
                "longitud_original": len(texto_original),
                "longitud_chunk": len(chunk_texto),
                
                # Texto del chunk (útil para debugging)
                "texto_chunk": chunk_texto[:500] + "..." if len(chunk_texto) > 500 else chunk_texto
            }
            
            points.append(models.PointStruct(
                id=str(uuid.uuid4()),
                vector=vector,
                payload=payload
            ))
            
            total_chunks += 1
            
        except Exception as e:
            errores.append({
                "fila": idx,
                "chunk": chunk_num,
                "error": str(e)[:200]
            })
            print(f"  Error fila {idx}, chunk {chunk_num}: {str(e)[:100]}")
            continue
    
    casos_procesados += 1
    
    # Progreso y rate limit
    if (idx + 1) % 50 == 0:
        print(f"  Procesados {casos_procesados} casos → {total_chunks} chunks creados")
    
    if idx % 10 == 0:
        time.sleep(0.2)  # Evitar rate limit


# ============================================
# SUBIDA A QDRANT
# ============================================
print(f"\n3. Subiendo {len(points)} vectores a Qdrant Cloud...")

BATCH_SIZE = 50
batches_subidos = 0

for i in range(0, len(points), BATCH_SIZE):
    batch = points[i : i + BATCH_SIZE]
    client_qdrant.upload_points(
        collection_name=COLLECTION_NAME,
        points=batch
    )
    batches_subidos += 1
    if batches_subidos % 5 == 0:
        print(f"  {batches_subidos} batches subidos ({i + len(batch)} vectores)")

# ============================================
# REPORTE FINAL
# ============================================
print(f"\n{'='*70}")
print(f"REPORTE FINAL DE INDEXACIÓN")
print(f"{'='*70}")
print(f"Casos procesados: {casos_procesados}/{len(df)}")
print(f"Total vectores creados: {total_chunks}")
print(f"Casos divididos en chunks: {len(casos_divididos)}")
print(f"Errores: {len(errores)}")
print(f"Promedio chunks/caso: {total_chunks/casos_procesados:.2f}")

if casos_divididos:
    print(f"\n{'='*70}")
    print(f"CASOS DIVIDIDOS (sin pérdida de información):")
    print(f"{'='*70}")
    for caso in casos_divididos:
        print(f"  • Fila {caso['fila']} - {caso['providencia']}")
        print(f"    └─ {caso['longitud_original']:,} chars → {caso['num_chunks']} chunks")

if errores:
    print(f"\n{'='*70}")
    print(f"ERRORES ENCONTRADOS:")
    print(f"{'='*70}")
    for error in errores:
        print(f"  • Fila {error['fila']}, Chunk {error['chunk']}: {error['error']}")

print(f"\n¡Colección '{COLLECTION_NAME}' completada con CERO pérdida de contexto!")

1. Reiniciando colección: prueba_tecnica_collection_cocu...
   -> Versión anterior eliminada.
   -> Colección creada vacía.

2. Procesando documentos con CHUNKING INTELIGENTE...
   - Max chars por chunk: 30,000
   - Overlap: 3,000 caracteres

   Procesados 50 casos → 50 chunks creados
   Procesados 100 casos → 100 chunks creados
   Procesados 150 casos → 150 chunks creados
   Procesados 200 casos → 200 chunks creados
   Procesados 250 casos → 250 chunks creados
   Procesados 300 casos → 300 chunks creados
   Fila 321 (SU.546/23): 2 chunks

3. Subiendo 330 vectores a Qdrant Cloud...
   5 batches subidos (250 vectores)

REPORTE FINAL DE INDEXACIÓN
Casos procesados: 329/329
Total vectores creados: 330
Casos divididos en chunks: 1
Errores: 0
Promedio chunks/caso: 1.00

CASOS DIVIDIDOS (sin pérdida de información):
  • Fila 321 - SU.546/23
    └─ 35,614 chars → 2 chunks

¡Colección 'prueba_tecnica_collection_cocu' completada con CERO pérdida de contexto!


In [ ]:
# ============================================
# SISTEMA RAG: BÚSQUEDA + GENERACIÓN
# ============================================

def buscar_casos_similares(consulta, top_k=10):
    """
    Busca casos similares en Qdrant usando búsqueda semántica.
    Deduplica automáticamente si hay chunks del mismo caso.
    """
    # 1. Convertir consulta en vector
    vector_consulta = get_embedding(consulta)
    
    # 2. Buscar en Qdrant (buscar más para compensar duplicados)
    resultados = client_qdrant.query_points(
        collection_name=COLLECTION_NAME,
        query=vector_consulta,
        limit=top_k * 3,  # Buscar 3x más para deduplicar
        with_payload=True
    ).points
    
    # 3. Deduplicar por id_caso (mantener el chunk más relevante)
    casos_unicos = {}
    for hit in resultados:
        id_caso = hit.payload.get('id_caso', hit.payload.get('fila_excel'))
        
        if id_caso not in casos_unicos:
            casos_unicos[id_caso] = hit
        elif hit.score > casos_unicos[id_caso].score:
            # Si encontramos un chunk mejor del mismo caso, reemplazar
            casos_unicos[id_caso] = hit
    
    # 4. Retornar top_k únicos
    return list(casos_unicos.values())[:top_k]


def buscar_casos_hibridos(consulta, keywords=None, top_k=10):
    """
    Búsqueda híbrida: combina búsqueda semántica con filtrado por palabras clave.
    Útil para acrónimos o términos técnicos específicos (ej: PIAR).
    
    Args:
        consulta: Pregunta en lenguaje natural
        keywords: Lista de palabras clave que DEBEN aparecer (ej: ['PIAR'])
        top_k: Número de resultados a retornar
    
    Returns:
        Lista de casos que coinciden con la búsqueda semántica Y contienen las keywords
    """
    # 1. Búsqueda semántica normal
    vector_consulta = get_embedding(consulta)
    
    # 2. Si hay keywords, buscar más resultados para compensar el filtrado
    limite = top_k * 20 if keywords else top_k * 3
    
    resultados = client_qdrant.query_points(
        collection_name=COLLECTION_NAME,
        query=vector_consulta,
        limit=limite,
        with_payload=True
    ).points
    
    # 3. Filtrar por keywords si se especificaron
    if keywords:
        resultados_filtrados = []
        for hit in resultados:
            # Buscar keywords en síntesis, resuelve o tema
            texto_completo = (
                str(hit.payload.get('sintesis', '')) + " " +
                str(hit.payload.get('resuelve', '')) + " " +
                str(hit.payload.get('tema', '')) + " " +
                str(hit.payload.get('texto_chunk', ''))
            ).upper()
            
            # Verificar si TODAS las keywords están presentes
            if all(kw.upper() in texto_completo for kw in keywords):
                resultados_filtrados.append(hit)
        
        resultados = resultados_filtrados
    
    # 4. Deduplicar por id_caso (mantener el chunk más relevante)
    casos_unicos = {}
    for hit in resultados:
        id_caso = hit.payload.get('id_caso', hit.payload.get('fila_excel'))
        
        if id_caso not in casos_unicos:
            casos_unicos[id_caso] = hit
        elif hit.score > casos_unicos[id_caso].score:
            casos_unicos[id_caso] = hit
    
    return list(casos_unicos.values())[:top_k]


def responder_consulta_legal(pregunta, num_casos=15, verbose=True):
    """
    Sistema RAG completo: Busca casos relevantes y genera respuesta en lenguaje coloquial.
    Detecta automáticamente términos técnicos (PIAR, etc.) y usa búsqueda híbrida.
    """
    # Detectar términos técnicos que requieren búsqueda híbrida
    terminos_tecnicos = {
        'PIAR': ['PIAR'],
        'acoso escolar': ['acoso', 'escolar'],
        'redes sociales': ['redes sociales', 'red social', 'Instagram', 'Facebook', 'Twitter', 'WhatsApp']
    }
    
    keywords_detectadas = []
    pregunta_upper = pregunta.upper()
    
    # Detectar PIAR específicamente
    if 'PIAR' in pregunta_upper:
        keywords_detectadas = ['PIAR']
        if verbose:
            print(f"Búsqueda HÍBRIDA (detectado: PIAR) para: '{pregunta}'")
            print(f"  Filtrando por keyword: PIAR")
    else:
        if verbose:
            print(f"Buscando casos relevantes para: '{pregunta}'")
    
    # 1. Buscar casos relevantes (híbrido si se detectó keyword)
    if keywords_detectadas:
        casos = buscar_casos_hibridos(pregunta, keywords=keywords_detectadas, top_k=num_casos)
    else:
        casos = buscar_casos_similares(pregunta, top_k=num_casos)
    
    if verbose:
        print(f"  {len(casos)} casos encontrados\n")
    
    # 2. Construir contexto con los casos encontrados
    contexto = "CASOS LEGALES RELEVANTES:\n\n"
    for i, caso in enumerate(casos, 1):
        payload = caso.payload
        contexto += f"--- CASO {i} (Relevancia: {caso.score:.2%}) ---\n"
        contexto += f"Providencia: {payload.get('providencia', 'N/A')}\n"
        contexto += f"Fecha: {payload.get('fecha', 'N/A')}\n"
        contexto += f"Tema: {payload.get('tema', 'N/A')}\n"
        contexto += f"Síntesis del caso: {payload.get('sintesis', 'N/A')}\n"
        contexto += f"Sentencia: {payload.get('resuelve', 'N/A')}\n"
        
        # Info adicional si es chunk
        if payload.get('es_chunk'):
            contexto += f"[Documento largo - Chunk {payload.get('chunk_num')}/{payload.get('total_chunks')}]\n"
        
        contexto += "\n"
    
    # 3. Prompt para GPT (respuesta en lenguaje coloquial)
    prompt = f"""Eres un asesor legal especializado en explicar casos judiciales de forma clara y sencilla a personas sin conocimientos legales.

Tu tarea es responder la siguiente pregunta basándote ÚNICAMENTE en los casos legales proporcionados.

PREGUNTA DEL CLIENTE:
{pregunta}

{contexto}

INSTRUCCIONES IMPORTANTES:
- Responde en lenguaje coloquial, como si hablaras con un amigo que NO sabe de leyes
- Evita términos jurídicos complejos (o explícalos de forma muy simple)
- Si hay varios casos, resúmelos de forma clara y estructurada
- Si preguntan por sentencias, explica qué se decidió de manera comprensible
- Si NO encuentras información relevante en los casos, dilo claramente
- Sé preciso pero comprensible
- Usa ejemplos o analogías si ayuda a la claridad

RESPUESTA EN LENGUAJE COLOQUIAL:"""
    
    # 4. Generar respuesta con GPT
    respuesta = client_openai.chat.completions.create(
        model="gpt-4.1-mini",  # Rápido y económico
        messages=[
            {"role": "system", "content": "Eres un asesor legal que explica casos complejos en lenguaje simple y accesible."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,  # Bajo para ser más preciso
        max_tokens=2000
    )
    
    return {
        "respuesta": respuesta.choices[0].message.content,
        "casos_usados": casos,
        "num_casos": len(casos),
        "relevancia_promedio": sum(c.score for c in casos) / len(casos) if casos else 0
    }

print("Funciones RAG configuradas correctamente")

Funciones RAG configuradas correctamente


In [24]:
# ============================================
# VERIFICACIÓN: Casos con PIAR en el dataset
# ============================================

print(" Verificando casos con PIAR en el dataset original...\n")

# Buscar en el DataFrame
casos_piar = df[df['texto_vectorizar'].str.contains('PIAR', case=False, na=False)]
print(f" Casos con 'PIAR' encontrados: {len(casos_piar)}")

if len(casos_piar) > 0:
    print(f"\n Detalles de los casos con PIAR:\n")
    for idx, row in casos_piar.iterrows():
        print(f"   Fila {idx}:")
        print(f"   - Providencia: {row.get('Providencia', 'N/A')}")
        print(f"   - Tema: {row.get('Tema - subtema', 'N/A')[:100]}...")
        
        # Buscar contexto donde aparece PIAR
        texto = str(row['texto_vectorizar'])
        pos_piar = texto.upper().find('PIAR')
        if pos_piar > 0:
            inicio = max(0, pos_piar - 150)
            fin = min(len(texto), pos_piar + 150)
            contexto = texto[inicio:fin]
            print(f"   - Contexto: ...{contexto}...")
        print()
else:
    print(" No se encontraron casos con PIAR en el DataFrame")

print("\n" + "="*70)
print("Verificación completada")
print("="*70)


 Verificando casos con PIAR en el dataset original...

 Casos con 'PIAR' encontrados: 2

 Detalles de los casos con PIAR:

   Fila 99:
   - Providencia: T-249/24
   - Tema: No disponible...
   - Contexto: ...para contener la precitada circunstancia y para proteger a la adolescente. Igualmente, porque trazaron un Plan Individualizado de Ajustes Razonables (PIAR) que no tuvo en cuenta las condiciones y necesidades académicas especiales que requería la joven, sino que se diseñó únicamente con el propósito ...

   Fila 204:
   - Providencia: T-085/23
   - Tema: ACCESIBILIDAD Y ADAPTABILIDAD COMO COMPONENTES ESENCIALES DEL DERECHO A LA EDUCACION CARENCIA ACTUAL...
   - Contexto: ...que realizó la institución para los niños y niñas de transición del aula regular y no se le había realizado el Plan Individual de Ajustes Razonables (PIAR). Se resalta el hecho de que el niño recuperó sus habilidades de comunicación y fue traslado a un aula regular de la institución. Frente a la pre...


Verifica

In [ ]:
import time
from datetime import datetime

# ============================================
# RESPONDER PREGUNTAS DEL CASO DE NEGOCIO
# ============================================

preguntas = [

    "¿Existen casos que hablan sobre el PIAR? Indique de qué trataron los casos y cuáles fueron sus sentencias"
    "¿Cuáles son las sentencias de 3 demandas relacionadas con redes sociales?",
    "¿De qué se trataron esas 3 demandas anteriores relacionadas con redes sociales?",
    "¿Cuál fue la sentencia del caso que habla de acoso escolar?",
    "¿Diga el detalle de la demanda relacionada con acoso escolar?",]

print("="*80)
print(" ASESOR LEGAL IA - RESPUESTAS A CONSULTAS")
print("="*80)
print(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Base de datos: {COLLECTION_NAME} ({330} vectores, 329 casos)\n")

resultados_finales = []

for i, pregunta in enumerate(preguntas, 1):
    print(f"\n{'='*80}")
    print(f"PREGUNTA {i}")
    print(f"{'='*80}")
    print(f" {pregunta}\n")
    
    # Generar respuesta
    resultado = responder_consulta_legal(pregunta, num_casos=15, verbose=True)
    
    print(f"RESPUESTA:")
    print("-" * 80)
    print(resultado["respuesta"])
    print("-" * 80)
    print(f"\nRelevancia promedio: {resultado['relevancia_promedio']:.2%}")
    print(f" Casos consultados: {resultado['num_casos']}")
    
    # Mostrar casos usados
    print("\n Casos más relevantes:")
    for j, caso in enumerate(resultado["casos_usados"][:3], 1):
        print(f"   {j}. {caso.payload.get('providencia')} - Relevancia: {caso.score:.2%}")
    
    # Guardar para reporte
    resultados_finales.append({
        "pregunta": pregunta,
        "respuesta": resultado["respuesta"],
        "relevancia": f"{resultado['relevancia_promedio']:.2%}",
        "casos": [
            {
                "providencia": c.payload.get('providencia'),
                "relevancia": f"{c.score:.2%}",
                "tema": c.payload.get('tema'),
                "fecha": c.payload.get('fecha')
            }
            for c in resultado["casos_usados"]
        ]
    })
    
    # Pausa para evitar rate limit
    time.sleep(1)

print(f"\n\n{'='*80}")
print("TODAS LAS PREGUNTAS RESPONDIDAS")
print(f"{'='*80}\n")

 ASESOR LEGAL IA - RESPUESTAS A CONSULTAS
Fecha: 2026-01-28 23:45:49
Base de datos: prueba_tecnica_collection_cocu (330 vectores, 329 casos)


PREGUNTA 1
 ¿Existen casos que hablan sobre el PIAR? Indique de qué trataron los casos y cuáles fueron sus sentencias

 Búsqueda HÍBRIDA (detectado: PIAR) para: '¿Existen casos que hablan sobre el PIAR? Indique de qué trataron los casos y cuáles fueron sus sentencias'
  Filtrando por keyword: PIAR
   2 casos encontrados

 RESPUESTA:
--------------------------------------------------------------------------------
Claro, te explico de forma sencilla lo que dicen dos casos importantes sobre el PIAR, que es el Plan Individual de Ajustes Razonables. Este plan es como un “plan personalizado” que hacen las escuelas para ayudar a estudiantes que tienen necesidades especiales, para que puedan aprender y participar en la escuela en igualdad de condiciones.

1. **Primer caso (Providencia T-085/23):**  
Aquí se trató de un niño con una discapacidad auditiva